In [12]:
from vietocr.tool.config import Cfg
from vietocr.model.trainer import Trainer
from vietocr.tool.predictor import Predictor
import matplotlib.pyplot as plt
from PIL import Image

In [7]:
config = Cfg.load_config_from_name("vgg_seq2seq")

In [ ]:
dataset_params = {
    "name":"name_dtset",
    "dataroot" : "C:\Data...",
    "train_annotation" : "label.txt",
    "valid_annotation": "label2.txt"
}
params = {
    "print_every" : 200,
    "valid_every" : 15*200,
    "iters":20000,
    "checkpoint":"./checkpoint/seq2seqocr_checkpoint.pth",
    "export":"./weigths/seq2seqocr.pth",
    "metrics":10000
}
config["trainer"].update(params)
config["dataset"].update(dataset_params)
config["device"] = "cuda:0"
config["dataloader"]["num_workers"] = 0

trainer = Trainer(config, pretrained=True)
trainer.train()

## `Inference`

In [ ]:
config = Cfg.load_config_from_name("vgg_transformer")
config["weights"] = "./seq2seqocr.pth"
config["cnn"]["pretrained"] = False
config["device"] = "cpu"
detector = Predictor(config)